In [150]:
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
from scipy import stats
import scipy as sp
import datetime as dt
import graspy as gp
import collections
import community
import simpy
import random
from graspy.simulations import sbm
from graspy.plot import heatmap

import cmocean as cmo
from collections import Counter
from numpy.random import normal, poisson, power

import community
import warnings; warnings.simplefilter('ignore')

%matplotlib inline

In [299]:
def utility_function_CES(good_vector, preferences, sigma = 5):
    '''
    To begin with we consider a simple CES utility function where alpha_i = alpha_j for all j and i such that
    sum alpha_i = 1
    '''
    N = len(good_vector)
    #alpha = 1/N
    utility_vector = [good_vector[i]**((sigma-1)/sigma)*preferences[i]**(1/sigma) for i in range(N)]
    utility = sum(utility_vector)**(sigma/(sigma-1))
    return utility

def utility_function_log(good_vector, preferences):
    '''
    To begin with we consider a simple CES utility function where alpha_i = alpha_j for all j and i such that
    sum alpha_i = 1
    '''
    utility = 0
    for i, good in enumerate(good_vector):
        if good != 0:
            utility += np.log(good)*preferences[i]
    return utility


def evaluate_trade(G, buyer, seller, interval = 1):
    '''
    Calculates the maximum utility of all possible trades between two trading partners
    Simplified such that it is known that alpha = alpha
    '''
    N = G.number_of_nodes()
    
    seller_goods = list(nx.get_node_attributes(G, 'goods')[seller].values())
    buyer_goods = list(nx.get_node_attributes(G, 'goods')[buyer].values())
    
    output = {'max_utility': 0, 'optimal_s_good':0, 'optimal_b_good':0,'optimal_q': 0,'optimal_p':0}
    # max_utility = utility_function(seller_goods)
    # Empty goods should be ignored
    for buyer_good in range(N):
        if buyer_goods[buyer_good] == 0:
            continue
        buyer_preferences = list(nx.get_node_attributes(G, 'preferences')[buyer])
        current_buyer_utility = utility_function_log(buyer_goods, buyer_preferences)
        for seller_good in range(N):
            if seller_goods[seller_good] == 0:
                continue
            
            seller_goods = list(nx.get_node_attributes(G, 'goods')[seller].values())
            buyer_goods = list(nx.get_node_attributes(G, 'goods')[buyer].values())
            
            relative_price = ((seller_goods[buyer_good] + buyer_goods[buyer_good])/
                            (seller_goods[seller_good] + buyer_goods[seller_good]))
            
            
            
            #max_q = seller_goods[seller_good]
            seller_preferences = list(nx.get_node_attributes(G, 'preferences')[seller])
            current_seller_utility = utility_function_log(seller_goods, seller_preferences)

            
            for q in range(1,101):
                if buyer_goods[buyer_good] - q*relative_price < 0:
                    break
                if seller_goods[seller_good] - q < 0:
                    break
                seller_goods[seller_good] -= q
                seller_goods[buyer_good] += q*relative_price
                
                buyer_goods[seller_good] += q
                buyer_goods[buyer_good] -= q*relative_price
                if utility_function_log(seller_goods, seller_preferences) - current_seller_utility < 0:
                    continue
                utility = utility_function_log(buyer_goods, buyer_preferences)
                if utility - current_buyer_utility > 0:
                    current_buyer_utility = utility
                    output['max_utility'] = utility
                    output['optimal_s_good'] = seller_good
                    output['optimal_b_good'] = buyer_good
                    output['optimal_q'] = q
                    output['optimal_p'] = relative_price
                
    return output

def make_trade(G, trade_info):
    '''
    Makes the buyers's optimal trade
    '''
    good_distribution = nx.get_node_attributes(G, 'goods')
    
    initial_ss_good = good_distribution[trade_info['seller']][trade_info['optimal_s_good']]
    initial_sb_good = good_distribution[trade_info['seller']][trade_info['optimal_b_good']]
    new_ss_good = initial_ss_good - trade_info['optimal_q']
    new_sb_good = initial_sb_good + trade_info['optimal_q']*trade_info['optimal_p']
    good_distribution[trade_info['seller']][trade_info['optimal_s_good']] = new_ss_good
    good_distribution[trade_info['seller']][trade_info['optimal_b_good']] = new_sb_good
    
    initial_bs_good = good_distribution[trade_info['buyer']][trade_info['optimal_s_good']]
    initial_bb_good = good_distribution[trade_info['buyer']][trade_info['optimal_b_good']]
    new_bs_good = initial_bs_good + trade_info['optimal_q']
    new_bb_good = initial_bb_good - trade_info['optimal_q']*trade_info['optimal_p']
    good_distribution[trade_info['buyer']][trade_info['optimal_s_good']] = new_bs_good
    good_distribution[trade_info['buyer']][trade_info['optimal_b_good']] = new_bb_good
    
    nx.set_node_attributes(G, good_distribution, 'goods')

def trade_qeue(G, N_iter = 10, random_order = True):
    H = G.copy()
    N = H.number_of_nodes()
    buyer_list = [i for i in range(N)]
    for qeue_counter in range(N_iter):
        if random_order == True:
            random.shuffle(buyer_list)
            print(buyer_list)
        for buyer in buyer_list:
            trade_to_make = {}
            seller_list = nx.neighbors(H, buyer)
            max_util = 0
            for seller in seller_list:
                if seller == buyer:
                    continue
                optimal_trade = evaluate_trade(H, buyer, seller)
                if optimal_trade['max_utility'] - max_util > 0:
                    max_util = optimal_trade['max_utility']
                    trade_to_make = optimal_trade
                    trade_to_make['seller'] = seller
                    trade_to_make['buyer'] = buyer
            if max_util == 0:
                continue
            make_trade(H, trade_to_make)
            #print("trade "+str(buyer), dt.datetime.now())
        print('qeue lap ' + str(qeue_counter), dt.datetime.now())
    return H

def set_goods(G, amount_list = [], random = False):
    '''
    Set the initial distribution of goods across the network
    Parameters:
    ----------
    G - network of trading partners
    amount_list - predetermined goods distribution (should be a dict)
    random - Boolean determining of the distribution should be random (not implemented)
    Returns:
    -------
    Nothing
    '''
    N = len(G)
    if amount_list != []:
        goods = dict(zip(nodes, amount_list))
    goods_list = []
    goods_index = [i for i in range(N)]
    node_index = goods_index # These are the same as node i starts with one unit of good i
    for i in range(N):
        amount_list = [0]*N
        amount_list[i] = 100
        goods = dict(zip(node_index, amount_list))
        goods_list.append(goods)
    initial_good_distribution = dict(zip(node_index, goods_list))
    nx.set_node_attributes(G, initial_good_distribution, 'goods')
    

In [300]:
def hamiltonian(preference, h_soc, h_i, alpha = .5):
    '''The function which every node minimise when they update their preferences'''
   
    H = (-alpha*np.cos((np.pi/2)*(preference - h_soc)) - (1-alpha)*np.cos((np.pi/2)*(preference-h_i)))
   
    #H = -alpha*preference*h_soc - beta*preference*h_i - (1 - alpha - beta)*preference*h_env
   
    return H


def set_initial_preferences(social_network, community_list = None, community_preferences = None, 
                            normal_dist=True):
    """
    Sets the preferences of the initial network based on community
    Inputs:
    social_network: network of nodes
    community_list: list of community sizes
    conn_within: list of initial preferences for each community
    normal_dist: whether or not the preferences should be a Gaussian-distributed value
                around the mean value of preferences for each community
    """
    
    N = social_network.number_of_nodes()
    
    preferences = dict([(i, 0) for i in range(N)])
    
    for node in range(N):
        if normal_dist == True:
            preferences[node] = np.random.normal(1/N,0.001, size = N)
            for pref in preferences[node]:
                if pref >= 1:
                    pref = 1
                elif pref < 0:
                    pref = 0
        else:
            preferences[node] = community_preferences[comm_num]

    nx.set_node_attributes(social_network, preferences, 'preferences')

def set_alphas(social_network, community_list, community_alphas, community_betas, normal_dist='False'):
    """
    Sets the alphas and betas of the initial network based on community
    Inputs:
    social_network: network of nodes
    community_list: list of community sizes
    community_alphas: list of community alphas
    community_betas: list of community betas
    normal_dist: whether or not the preferences should be a Gaussian-distributed value
                around the mean value of preferences for each community
                
    Notes
    for the normal_dist='False' case more code should be added to ensure beta + alpha <= 1
    """
        
    alphas = dict([(i, 0) for i in range(sum(community_list))])
    counter = 0
    
    for comm_num in range(len(community_list)):
        for node in range(community_list[comm_num]):
            if normal_dist=='True':
                alphas[counter] = np.random.normal(community_alphas[comm_num],0.2)
                betas[counter] = np.random.normal(community_betas[comm_num],0.2)
                if alphas[counter] >= 1:
                    alphas[counter] = 0.9999
                elif alphas[counter] <= 0:
                    alphas[counter] = 0
                if betas[counter] >= 1:
                    betas[counter] = 0.9999
                elif betas[counter] <= 0:
                    betas[counter] = 0
            else:
                alphas[counter] = community_alphas[comm_num]
                betas[counter] = community_betas[comm_num]
            counter = counter + 1

    nx.set_node_attributes(social_network, alphas, 'alpha')

In [481]:
def update_preference(social_network, node, first_update, method = 'voter rule', alpha = 0.5, 
                      nu = 100):
    '''
    Update preference of a node in the graph using specified method
    inputs:
    social_network: Graph
    node: node in Graph
    method: social updating rule ('majority rule' or 'voter rule')
    alpha: parameter governing to what extent individuals are influenced by their neighbors when updating as 
    opposed to environment and individual preference
    beta: parameter governing to what extent individuals are influenced by their individual preference 
    as opposed to environment and neighbors
    alpha, beta in [0,1] and alpha + beta <= 1
    output:
    the new preference, node takes on this preference
    '''
    gamma = 0.2 # This should perhaps not be hardcoded
    h_soc = 0
    
    old_preference = social_network.node[node]['preferences']
    neighbors = nx.all_neighbors(social_network, node)
    preference_list = [social_network.node[i]['preferences'] for i in neighbors]
    
    if method == 'majority rule':
        h_soc = np.average(preference_list)
    elif method == 'voter rule':
        h_soc = preference_list[np.random.random_integers(0,len(preference_list) - 1)]
    elif method == 'expert rule':
        if first_update == True:
            old_preference = preference_list[np.random.random_integers(0,len(preference_list) - 1)]
        h_soc = old_preference
    elif method == 'max rule':
        pos_max = np.max(preference_list)
        neg_max = np.min(preference_list)
        if np.abs(pos_max) > np.abs(neg_max):
            h_soc = pos_max
        else:
            h_soc = neg_max
    elif method == 'min rule':
        pos_list = [i for i in preference_list if i >= 0]
        neg_list = [i for i in preference_list if i <= 0]
        if len(pos_list) != 0:
            pos_min = np.min(pos_list)
        else:
            pos_min = 1
        if len(neg_list) != 0:
            neg_min = np.max(neg_list)
        else:
            neg_min = -1
        if abs(pos_min) >= abs(neg_min):
            h_soc = neg_min
        else:
            h_soc = pos_min
    else:
        print('Invalid preference updating rule')
    
    new_preference = h_soc
    dH = (hamiltonian(new_preference, h_soc, h_i=old_preference, alpha=alpha) 
          - hamiltonian(old_preference, h_soc, h_i=old_preference, alpha=alpha))
    
    for i in dH:
        if i < 0:
            social_network.node[node]['preferences'] = new_preference
        elif np.random.rand() < 1/(1+np.exp(nu*i)):
            social_network.node[node]['preferences'] = new_preference

def generate_clocks(social_network, env, interval, method = 'voter rule', alpha=0.5, 
                    distribution = 'none'):
    """
    Generates clocks for every node in the network
    Inputs:
    env: SimPy environment
    interval: Parameter setting how often clocks update
    G: the network which is being simulated
    
    Note:
    Beta is hardcoded to be (1-alpha)/2
    """
        
    if method == 'chaos rule':
        method_list = (['voter rule', 'expert rule', 'majority rule', 'max rule', 'min rule']*
                       int(np.floor(len(social_network.node())/5)))
        while len(method_list) != len(social_network.node()):
            method_list.append(method_list[random.randint(0,4)])
        for i in social_network.node():
            variation = variation_generation(distribution)
            assign_method = method_list.pop(random.randint(0,len(method_list)-1))
            if variation + alpha > 1:
                c = poisson_clock(social_network, env, i, interval, assign_method, alpha = 1)
            elif variation + alpha < 1:
                c = poisson_clock(social_network, env, i, interval, assign_method, alpha = 0)
            else:
                c = poisson_clock(social_network, env, i, interval, assign_method, 
                                  alpha = alpha + variation)
            env.process(c)
    else:
        for i in social_network.node():
            variation = variation_generation(distribution)
            if variation + alpha > 1:
                cp = preference_clock(social_network, env, i, interval, method, alpha = 1)
                ct = trade_clock(social_network, env, i, interval)
            elif variation + alpha < 1:
                cp = preference_clock(social_network, env, i, interval, method, alpha = 0)
                ct = trade_clock(social_network, env, i, interval)
            else:
                cp = preference_clock(social_network, env, i, interval, method, alpha = alpha + variation)
                ct = trade_clock(social_network, env, i, interval)
            env.process(cp)
            env.process(ct)

def variation_generation(distribution):
    '''
    Parameters:
    distribution (string) : Specifies which distribution variation is drawn from. Can be:
        'normal', mean = 0, StDev = 1
        'left skew', mean = , StDev = 1, skewness =
        'right skew', mean = 0, StDev = 1, skewness = 
        'none' => variation = 0
    Returns:
    variation (float) : a draw from one of the distributions above or 0
    '''
    mean = 0
    stdev = 0.2
    skew = 1
    
    if distribution == 'normal':
        variation = np.random.normal()
    elif distribution == 'left skew':
        variation = sp.stats.skewnorm.rvs(-skew, scale = stdev)
    elif distribution == 'right skew':
        variation = sp.stats.skewnorm.rvs(skew, scale = stdev)
    else:
        variation = 0
    return variation

def preference_clock(social_network, env, name, interval, method = 'voter rule', alpha = 0.5):
    """
    A clock dictating how often a node updates its preferences
    Inputs:
    env: SimPy environment
    name: the index of the node
    interval: Parameter setting how often clocks update
    G: the network which is being simulated
    """
    first = True
    first_update = True
    while True:
        if first == False:
            update_preference(social_network, name, first_update, method, alpha = alpha)
            pref_changes = nx.get_node_attributes(social_network, 'pref_changes')
            pref_changes[name] += 1
            nx.set_node_attributes(social_network, pref_changes, 'pref_changes')
#            print('At time %7.4f Updating node: %s ' % (env.now, name))
            first_update = False
        first = False
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)
        
def trade_clock(social_network, env, name, interval):
    """
    A clock dictating how often a node updates its preferences
    Inputs:
    env: SimPy environment
    name: the index of the node
    interval: Parameter setting how often clocks update
    G: the network which is being simulated
    """
    first = True
    first_update = True
    buyer = name
    N = nx.number_of_nodes(social_network)
    while True:
        if first == False:
            trade_to_make = {}
            seller_list = nx.neighbors(social_network, buyer)
            max_util = 0
            for seller in seller_list:
                if seller == buyer:
                    continue
                optimal_trade = evaluate_trade(social_network, buyer, seller)
                if optimal_trade['max_utility'] - max_util > 0:
                    max_util = optimal_trade['max_utility']
                    trade_to_make = optimal_trade
                    trade_to_make['seller'] = seller
                    trade_to_make['buyer'] = buyer
            if max_util == 0:
                continue
            make_trade(social_network, trade_to_make)
            trades = nx.get_node_attributes(social_network, 'trades')
            trades[name] += 1
            nx.set_node_attributes(social_network, trades, 'trades')
            utilities = []
            for i in range(N):
                gs = list(nx.get_node_attributes(G, 'goods')[i].values())
                uti = utility_function_log(gs, list(nx.get_node_attributes(G, 'preferences').values())[i])
                utilities.append(uti)
            utility = {i:u for i,u in enumerate(utilities)}
            nx.set_node_attributes(social_network, utility, 'utility')
            
#print('At time %7.4f Updating node: %s ' % (env.now, name))
            first_update = False
        first = False
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)

In [489]:
def preference_dynamic_simulation(social_network, interval, method, alpha, distribution, 
                              timestep, timesteps, draw_network = False, return_network = False):
    '''
    Function that runs the preference Dynamic on a Social Network simulation based on specified parameters and 
    returns data from the simulation recorded at every timestep
    
    Parameters
    social_network: Graph
    interval: parameters setting how often clocks update
    method: social updating rule ('majority rule' or 'voter rule')
    alpha: parameter governing to what extent individuals are influenced by their neighbors when updating as 
    opposed to environment and individual preference
    beta: parameter governing to what extent individuals are influenced by their individual preference as 
    opposed to environment and neighbors
    alpha, beta in [0,1] and alpha + beta <= 1
    distribution: sets the distribution of variation around alpha and beta
    community_list: list of community sizes
    disaster_communities: Specifies the harshness off the disaster at a community level
    draw_network: specifies whether a graph should be drawn at the beginning of the simulation, before the 
    disaster, after the disaster and at the end of the simulation
    
    Returns
    gini_data: panda DataFrame containing the gini coefficient of the belifs at each timestep. Divided up into
    3 columns before disaster, during disaster and after disaster
    average_data: like gini_data but contains the average preferences
    
    Notes
    There are a lot more parameters that can be specified in the underlaying functions already included in here
    The function can be extended to also generating the social network if we want to play around with network 
    topology more easily
    
    '''
    
    print('Starting at: ', dt.datetime.now())
    G = social_network.copy()
    
    # Setup and start the simulation
    env = simpy.Environment()
    
    N = G.number_of_nodes()
    
    init_pref = list(nx.get_node_attributes(G, 'preferences').values())
    init_good = list(nx.get_node_attributes(G, 'goods').values())
    
    utilities = []
    for i in range(N):
        gs = list(nx.get_node_attributes(G, 'goods')[i].values())
        uti = utility_function_log(gs, init_pref[i])
        utilities.append(uti)
    utility = {i:u for i,u in enumerate(utilities)}
    
    
    nx.set_node_attributes(G, 0, 'trades')
    nx.set_node_attributes(G, 0, 'pref_changes')
    nx.set_node_attributes(G, utility, 'utility')
    
    # Start processes and run
    generate_clocks(G, env, interval, alpha=alpha, method=method, distribution = False)
    
    g_gini_data = np.zeros((timesteps+1, N))
    pref_data = np.zeros((N, N, timesteps+1))
    good_data = np.zeros((N, N, timesteps+1))
    util_data = np.zeros((timesteps+1, N))
    u_gini_data = []
    
    pref_data[:,:,0] = np.array(init_pref)
    
    for i,good in enumerate(init_good):
        g = list(good.values())
        good_data[i, :, 0] = g
    
    util_data[0, :] = np.array(utilities)
    u_gini_data.append(gini2(utilities))
    
#    print('Beginning of Simulation')
    if draw_network == True:
        draw_preferences(G)
        
    for t in range(1,timesteps+1):
        if t%5 == 0:
            print('timestep ', str(t*timestep), ' of ', str(timesteps*timestep))
        env.run(until=(t)*timestep)
        goods = list(nx.get_node_attributes(G, 'goods').values())
        preferences = list(nx.get_node_attributes(G, 'preferences').values())
        utility = list(nx.get_node_attributes(G, 'utility').values())
        #gini_data[t, :] = gini(goods)
        for i,good in enumerate(goods):
            g = list(good.values())
            good_data[i, :, t] = g
        
        pref_data[:,:, t] = np.array(preferences)
        
        util_data[t, :] = np.array(utility)
        u_gini_data.append(gini2(utility))
        
        #fig_title = 'disaster' + str(t) + '.png'
        #draw_preferences(social_network,save='True',title=fig_title)

    
    
    total_trades = sum(list(nx.get_node_attributes(G, 'trades').values()))
    total_pref_changes = sum(list(nx.get_node_attributes(G, 'pref_changes').values()))
    
    data = {'goods':good_data, 'preferences': pref_data, 'utilities':util_data,'utility gini':u_gini_data,
            'trade volume':total_trades, 'pref changes':total_pref_changes}
    
    if draw_network == True:
        draw_preferences(G)
    
#    print('Final State')
    if draw_network == True:
        draw_preferences(G)
    print('Finished at: ', dt.datetime.now())
    if return_network == True:
        return G, data
    elif return_network == False:
        return data
    else:
        'Print specify return_network either True or False'
    
def gini(x):
    # (Warning: This is a concise implementation, but it is O(n**2)
    # in time and memory, where n = len(x).  *Don't* pass in huge
    # samples!)

    # Mean absolute difference
    mad = np.abs(np.subtract.outer(x, x)).mean()
    # Relative mean absolute difference
    rmad = mad/np.mean(x)
    # Gini coefficient
    g = 0.5 * rmad
    return g

def gini2(x, w=None):
    # Array indexing requires reset indexes.
    x = pd.Series(x).reset_index(drop=True)
    if w is None:
        w = np.ones_like(x)
    w = pd.Series(w).reset_index(drop=True)
    n = x.size
    wxsum = sum(w * x)
    wsum = sum(w)
    sxw = np.argsort(x)
    sx = x[sxw] * w[sxw]
    sw = w[sxw]
    pxi = np.cumsum(sx) / wxsum
    pci = np.cumsum(sw) / wsum
    g = 0.0
    for i in np.arange(1, n):
        g = g + pxi.iloc[i] * pci.iloc[i - 1] - pci.iloc[i] * pxi.iloc[i - 1]
    return g

def preference_gini_average(social_network):
    preference_data = social_network.nodes.data()
    preferences = []
    for node in preference_data:
        preferences.append((1 + node[1]['preferences'])/2)
    return np.average(preferences)

def community_average(social_network):
    preference_data = social_network.nodes.data()
    average_preferences_by_comm = []
    counter = 1
    for node in preference_data:
        preferences = []
        for i in range((counter-1)*10,10*counter):
            preferences.append((1 + node[1]['preferences'])/2)
        average_preferences_by_comm.append(np.average(preferences))
        counter += 1
        
    return average_preferences_by_comm


# Data we want
## Different network structures (all same number of nodes = 100)
Complete graph 

Barabasi Albert (preferential attachment)

Stochastic block model

How many iterations of each model? Maybe 10?

Variations on the models?

Preference change rule - stick to one for now

### We might want utility too! Could measure inequailty in utility


# Parameter sweep?

# Graphs we want
## Follow one agent over simulation
### Collect good data and preference data
## Gini of utility over time
## Utility of 10 agents over time

# Extensions
## Add option to produce good
## Right now agents may change their preferences such that their utility decreases
### Preference change should be independent of change in utility -> otherwise agents would just change their preferences such that they increase their utility makes no sense


In [490]:
G_ba = nx.barabasi_albert_graph(10, 4)
G = nx.complete_graph(10)
set_goods(G)
set_goods(G_ba)
set_initial_preferences(G)


In [491]:
interval = 5
method = 'voter rule'
alpha = 0.5
distribution = 'none'
timestep = 10
timesteps = 10

data = preference_dynamic_simulation(G_c, interval, method, alpha, distribution, 
                              timestep, timesteps)

Starting at:  2020-01-19 20:42:17.936572
timestep  50  of  100
timestep  100  of  100
Periods:  100
Finished at:  2020-01-19 20:42:36.955282


In [411]:
trades

221

In [244]:
set_goods(G_c)
set_goods(G_ba)
set_initial_preferences(G_c)

[0.10012342 0.10166883 0.10148411 0.09923944 0.10121008 0.09880909
 0.10110968 0.09918017 0.09964145 0.10014026]
[0.09924499 0.09971979 0.10089755 0.09908044 0.10120464 0.10125497
 0.1003538  0.09949857 0.09892894 0.10146558]
[0.10061062 0.0989716  0.10185661 0.10001138 0.09917386 0.10180568
 0.09899818 0.10036144 0.10095267 0.10120954]
[0.09924084 0.09960055 0.09890788 0.09984986 0.09875169 0.1011155
 0.09971729 0.10221244 0.1001588  0.09948824]
[0.10085299 0.10107558 0.10020766 0.10200368 0.0989926  0.09800398
 0.10068159 0.09963843 0.10160461 0.10066632]
[0.09861417 0.10155936 0.09962072 0.09957123 0.10288027 0.0995652
 0.09989582 0.09985083 0.09951991 0.10038815]
[0.10007229 0.10140265 0.09931059 0.09869391 0.1014532  0.09989052
 0.09979148 0.10127014 0.09992221 0.09924545]
[0.09946781 0.10092143 0.09988502 0.10067996 0.09832399 0.09908288
 0.10050103 0.10109574 0.10094131 0.10004139]
[0.09938161 0.10079656 0.10025412 0.10047268 0.10117561 0.09891238
 0.1002882  0.09943161 0.100480

In [245]:
G_t_c = trade_qeue(G_c, N_iter = 10)

[6, 7, 0, 4, 9, 1, 2, 5, 8, 3]
qeue lap 0 2020-01-16 21:02:10.039255
[1, 8, 9, 5, 2, 6, 4, 3, 7, 0]
qeue lap 1 2020-01-16 21:02:10.206027
[8, 7, 1, 4, 2, 0, 6, 5, 9, 3]
qeue lap 2 2020-01-16 21:02:10.536022
[0, 8, 9, 4, 7, 3, 1, 6, 5, 2]
qeue lap 3 2020-01-16 21:02:10.989190
[5, 3, 9, 4, 0, 6, 2, 7, 1, 8]
qeue lap 4 2020-01-16 21:02:11.642615
[3, 5, 1, 8, 9, 7, 0, 6, 2, 4]
qeue lap 5 2020-01-16 21:02:12.262628
[0, 4, 5, 3, 9, 6, 2, 8, 1, 7]
qeue lap 6 2020-01-16 21:02:13.104847
[6, 7, 8, 9, 1, 2, 4, 3, 0, 5]
qeue lap 7 2020-01-16 21:02:14.072290
[3, 4, 0, 5, 7, 9, 8, 1, 2, 6]
qeue lap 8 2020-01-16 21:02:15.139228
[7, 2, 9, 3, 0, 5, 6, 4, 8, 1]
qeue lap 9 2020-01-16 21:02:16.226888


In [207]:
G_t_ba = trade_qeue(G_ba, N_iter = 10)

[7, 9, 3, 0, 1, 5, 8, 2, 4, 6]
qeue lap 0 2020-01-16 18:27:58.338464
[0, 6, 5, 4, 3, 1, 8, 9, 2, 7]
qeue lap 1 2020-01-16 18:27:58.442491
[3, 5, 4, 1, 9, 8, 7, 6, 2, 0]
qeue lap 2 2020-01-16 18:27:58.558120
[0, 9, 3, 8, 4, 6, 5, 1, 7, 2]
qeue lap 3 2020-01-16 18:27:58.736611
[6, 9, 5, 0, 8, 4, 2, 1, 7, 3]
qeue lap 4 2020-01-16 18:27:59.028864
[0, 5, 7, 6, 2, 4, 3, 8, 9, 1]
qeue lap 5 2020-01-16 18:27:59.327790
[1, 9, 0, 3, 8, 7, 6, 5, 4, 2]
qeue lap 6 2020-01-16 18:27:59.785324
[8, 1, 7, 5, 3, 2, 6, 4, 9, 0]
qeue lap 7 2020-01-16 18:28:00.181778
[2, 4, 0, 3, 9, 5, 6, 7, 1, 8]
qeue lap 8 2020-01-16 18:28:00.575011
[9, 2, 0, 6, 5, 4, 7, 3, 1, 8]
qeue lap 9 2020-01-16 18:28:01.002473


In [2]:
def generate_social_network(community_list, conn_within = 1, conn_between = 0.1):
    """
    Generates a degree-corrected stochastic block model network
    Inputs:
    community_list: list of community sizes
    conn_within: likelihood of two nodes in the same community sharing an edge
    conn_between: likelihood of two nodes in different communities being connected
    """

    N_total = sum(community_list)
    num_comm = len(community_list)
    
    # Creates the connection matrix between communities
    p = np.full((num_comm,num_comm), conn_between) 
    p = p + np.diag([conn_within-conn_between for i in range(num_comm)])

    # Obtains the correct degree distribution from the Barabasi-Albert Graph
    BAG = nx.barabasi_albert_graph(N_total,1)
    degs = np.array(list(dict(BAG.degree()).values()))
    p_degs = degs/sum(degs)

#    plt.hist(p_degs)
#    plt.show()
#    plt.clf()
    
    G = sbm(n=community_list,p=p, dc=p_degs)
    
#    print(G)
    
    return G

In [3]:
def set_initial_preferences(social_network, community_list, community_preferences, normal_dist='False'):
    """
    Sets the preferences of the initial network based on community
    Inputs:
    social_network: network of nodes
    community_list: list of community sizes
    conn_within: list of initial preferences for each community
    normal_dist: whether or not the preferences should be a Gaussian-distributed value
                around the mean value of preferences for each community
    """
        
    preferences = dict([(i, 0) for i in range(sum(community_list))])
    counter = 0
    
    for comm_num in range(len(community_list)):
        for node in range(community_list[comm_num]):
            if normal_dist=='True':
                preferences[counter] = np.random.normal(community_preferences[comm_num],0.2)
                if preferences[counter] >= 1:
                    preferences[counter] = 0.9999
                elif preferences[counter] <= -1:
                    preferences[counter] = -1
            else:
                preferences[counter] = community_preferences[comm_num]
            counter = counter + 1

    nx.set_node_attributes(social_network, preferences, 'preference')

def set_alphas_betas(social_network, community_list, community_alphas, community_betas, normal_dist='False'):
    """
    Sets the alphas and betas of the initial network based on community
    Inputs:
    social_network: network of nodes
    community_list: list of community sizes
    community_alphas: list of community alphas
    community_betas: list of community betas
    normal_dist: whether or not the preferences should be a Gaussian-distributed value
                around the mean value of preferences for each community
                
    Notes
    for the normal_dist='False' case more code should be added to ensure beta + alpha <= 1
    """
        
    alphas = dict([(i, 0) for i in range(sum(community_list))])
    betas = dict([(i, 0) for i in range(sum(community_list))])
    counter = 0
    
    for comm_num in range(len(community_list)):
        for node in range(community_list[comm_num]):
            if normal_dist=='True':
                alphas[counter] = np.random.normal(community_alphas[comm_num],0.2)
                betas[counter] = np.random.normal(community_betas[comm_num],0.2)
                if alphas[counter] >= 1:
                    alphas[counter] = 0.9999
                elif alphas[counter] <= 0:
                    alphas[counter] = 0
                if betas[counter] >= 1:
                    betas[counter] = 0.9999
                elif betas[counter] <= 0:
                    betas[counter] = 0
            else:
                alphas[counter] = community_alphas[comm_num]
                betas[counter] = community_betas[comm_num]
            counter = counter + 1

    nx.set_node_attributes(social_network, alphas, 'alpha')
    nx.set_node_attributes(social_network, betas, 'beta')


        

In [4]:
def draw_preferences(social_network, save='False', title=''):
    """
    Draws the social network using a Kamada-Kawaii Layout
    Inputs:
    social_network: network of nodes
    """

    for i in range(len(social_network.nodes)):
        if social_network.nodes[i]['preference'] == 1:
            social_network.nodes[i]['preference'] = 0.9999
    
    node_colors = [plt.cm.viridis((1+social_network.nodes[i]['preference'])/2) for i in range(len(social_network))]
    
    #node_colors = [plt.cm.viridis(social_network.nodes[i]['preference']) for i in range(len(social_network))]
    
    #print(social_network.nodes[0])
    #print((1+social_network.nodes[0]['preference'])/2)
    
    fig, ax = plt.subplots(1,1,figsize=(10,9))

    pos = nx.kamada_kawai_layout(social_network)

    nx.draw_networkx_nodes(social_network, pos, node_color=node_colors, node_size=400, linewidths=2.0,
                           alpha=0.99, edgecolors='w', ax=ax)
    nx.draw_networkx_edges(social_network, pos, edge_color='#666666', width=4.5, alpha=0.5, ax=ax)

    sm = plt.cm.ScalarMappable(cmap=plt.cm.viridis, norm=plt.Normalize(0,1))
    sm._A = []
    plt.colorbar(sm)

    ax.set_axis_off()

    #ax.set_title("Nodes Colored by preference", fontsize=16)

    if save=='True':
        plt.savefig(title, dpi=425, bbox_inches='tight')

#    plt.show()
    
    plt.clf()

In [5]:
def draw_envs(social_network):
    """
    Draws the environmental factors for the network using a Kamada-Kawaii Layout
    Inputs:
    social_network: network of nodes
    """

    for i in range(len(social_network.nodes)):
        if social_network.nodes[i]['env'] == 1:
            social_network.nodes[i]['env'] = 0.9999
    
    node_colors = [plt.cm.viridis((1+social_network.nodes[i]['env'])/2) for i in range(len(social_network))]
    
    #node_colors = [plt.cm.viridis(social_network.nodes[i]['belief']) for i in range(len(social_network))]
    
    print(social_network.nodes[0])
    print((1+social_network.nodes[0]['env'])/2)
    
    fig, ax = plt.subplots(1,1,figsize=(10,9))

    pos = nx.kamada_kawai_layout(social_network)

    nx.draw_networkx_nodes(social_network, pos, node_color=node_colors, node_size=400, linewidths=2.0,
                           alpha=0.99, edgecolors='w', ax=ax)
    nx.draw_networkx_edges(social_network, pos, edge_color='#666666', width=4.5, alpha=0.5, ax=ax)

    sm = plt.cm.ScalarMappable(cmap=plt.cm.viridis, norm=plt.Normalize(-1,1))
    sm._A = []
    plt.colorbar(sm)

    ax.set_axis_off()

    ax.set_title("Nodes Colored by Environment", fontsize=16)

    #plt.savefig('e_coli.png', dpi=425, bbox_inches='tight')
    plt.show()

In [6]:
def hamiltonian(preference, h_soc, h_i, alpha = .5, beta = .25,  h_env = 0):
    '''The function which every node minimise when they update their preferences'''
   
    H = (-alpha*np.cos((np.pi/2)*(preference - h_soc)) - beta*np.cos((np.pi/2)*(preference-h_i)) 
         - (1-alpha-beta)*h_env*np.sin((np.pi/2)*preference))
   
    #H = -alpha*preference*h_soc - beta*preference*h_i - (1 - alpha - beta)*preference*h_env
   
    return H



def plot_network(social_network, env, values):
    while True:
        values = [val_map.get(social_network.node[node]['preference'], 0.25) for node in list(social_network)]
        nx.draw_networkx(social_network, node_color = values)
        yield env.timeout(5)


In [7]:
def preference_dynamic_simulation(social_network, interval, method, alpha, beta, distribution, 
                              community_list, disaster_communities, timestep, before_timesteps, 
                              duration_timesteps, after_timesteps, draw_network = False, return_network = False):
    '''
    Function that runs the preference Dynamic on a Social Network simulation based on specified parameters and 
    returns data from the simulation recorded at every timestep
    
    Parameters
    social_network: Graph
    interval: parameters setting how often clocks update
    method: social updating rule ('majority rule' or 'voter rule')
    alpha: parameter governing to what extent individuals are influenced by their neighbors when updating as 
    opposed to environment and individual preference
    beta: parameter governing to what extent individuals are influenced by their individual preference as 
    opposed to environment and neighbors
    alpha, beta in [0,1] and alpha + beta <= 1
    distribution: sets the distribution of variation around alpha and beta
    community_list: list of community sizes
    disaster_communities: Specifies the harshness off the disaster at a community level
    draw_network: specifies whether a graph should be drawn at the beginning of the simulation, before the 
    disaster, after the disaster and at the end of the simulation
    
    Returns
    gini_data: panda DataFrame containing the gini coefficient of the belifs at each timestep. Divided up into
    3 columns before disaster, during disaster and after disaster
    average_data: like gini_data but contains the average preferences
    
    Notes
    There are a lot more parameters that can be specified in the underlaying functions already included in here
    The function can be extended to also generating the social network if we want to play around with network 
    topology more easily
    
    '''
    if alpha + beta > 1:
        print('invalid alpha and beta, sum must be less than 1')
        return 
    
    G = social_network.copy()
    # Setup and start the simulation
    env = simpy.Environment()
    
    # Start processes and run
    generate_clocks(G,env,interval,alpha=alpha, beta = beta, method=method, 
                    distribution = False)
    
    gini_data = pd.DataFrame(columns = ['Before Disaster', 'During Disaster', 'After Disaster', 'All'])
    average_data = pd.DataFrame(columns = ['Before Disaster', 'During Disaster', 'After Disaster', 'All'])
    
#    print('Beginning of Simulation')
    if draw_network == True:
        draw_preferences(G)
        
    counter = 0
    for t in range(1, before_timesteps+1):
    #    print('timestep ', str(t), ' of ', str(num_timesteps))
        env.run(until=t*timestep)
        gini, average = preference_gini_average(G)
        gini_data.at[counter,'Before Disaster'] = gini
        average_data.at[counter,'Before Disaster'] = average
        gini_data.at[t-1,'All'] = gini
        average_data.at[t-1,'All'] = average
        #fig_title = 'disaster' + str(t) + '.png'
        #draw_preferences(social_network,save='True',title=fig_title)
        counter += 1
    
#    print('Before Disaster')
    
    if draw_network == True:
        draw_preferences(G)
    
    
    set_env(G,community_list,disaster_communities)
#    print('BAM!!!!')
#    print('DISASTER!!!!')
    
    counter = 0
    for t in range(before_timesteps + 1,before_timesteps + duration_timesteps + 1):
    #    print('timestep ', str(t), ' of ', str(num_timesteps))
        env.run(until=t*timestep)
        gini, average = preference_gini_average(G)
        gini_data.at[counter,'During Disaster'] = gini
        average_data.at[counter,'During Disaster'] = average
        gini_data.at[t-1,'All'] = gini
        average_data.at[t-1,'All'] = average
        #fig_title = 'disaster' + str(t) + '.png'
        #draw_preferences(social_network,save='True',title=fig_title)
    
        counter += 1
        
#    print('After Disaster')
    
    if draw_network == True:
        draw_preferences(G)    
    
    set_env(G,community_list,np.zeros(len(community_list)))
#    print('FEMA did its job')
    
    counter = 0
    for t in range(duration_timesteps+before_timesteps + 2, 2 + before_timesteps + duration_timesteps 
                   + after_timesteps):
    #    print('timestep ', str(t), ' of ', str(num_timesteps))
        env.run(until=t*timestep) 
        gini, average = preference_gini_average(G)
        gini_data.at[counter,'After Disaster'] = gini
        average_data.at[counter,'After Disaster'] = average
        gini_data.at[t-1,'All'] = gini
        average_data.at[t-1,'All'] = average
        #fig_title = 'disaster' + str(t) + '.png'
        #draw_preferences(social_network,save='True',title=fig_title)
        counter += 1
    
    
#    print('Final State')
    if draw_network == True:
        draw_preferences(G)
    if return_network == True:
        return G, gini_data, average_data
    elif return_network == False:
        return gini_data, average_data
    else:
        'Print specify return_network either True or False'
    
def gini(x):
    # (Warning: This is a concise implementation, but it is O(n**2)
    # in time and memory, where n = len(x).  *Don't* pass in huge
    # samples!)

    # Mean absolute difference
    mad = np.abs(np.subtract.outer(x, x)).mean()
    # Relative mean absolute difference
    rmad = mad/np.mean(x)
    # Gini coefficient
    g = 0.5 * rmad
    return g

def gini2(x, w=None):
    # Array indexing requires reset indexes.
    x = pd.Series(x).reset_index(drop=True)
    if w is None:
        w = np.ones_like(x)
    w = pd.Series(w).reset_index(drop=True)
    n = x.size
    wxsum = sum(w * x)
    wsum = sum(w)
    sxw = np.argsort(x)
    sx = x[sxw] * w[sxw]
    sw = w[sxw]
    pxi = np.cumsum(sx) / wxsum
    pci = np.cumsum(sw) / wsum
    g = 0.0
    for i in np.arange(1, n):
        g = g + pxi.iloc[i] * pci.iloc[i - 1] - pci.iloc[i] * pxi.iloc[i - 1]
    return g

def preference_gini_average(social_network):
    preference_data = social_network.nodes.data()
    preferences = []
    for node in preference_data:
        preferences.append((1 + node[1]['preference'])/2)
    return gini2(preferences), np.average(preferences)

def community_average(social_network):
    preference_data = social_network.nodes.data()
    average_preferences_by_comm = []
    counter = 1
    for node in preference_data:
        preferences = []
        for i in range((counter-1)*10,10*counter):
            preferences.append((1 + node[1]['preference'])/2)
        average_preferences_by_comm.append(np.average(preferences))
        counter += 1
        
    return average_preferences_by_comm


In [103]:
np.log(-1)  np.log(1)

False